COMP5930M Scientific Computation - Coursework 2
Template for the Burgers equation

In [1]:
import numpy as np
from numpy.linalg import norm

def myNewtonSys(fnon, jac, x0, tol, maxk, *fnonargs):
    # fnon     - name of the nonlinear function f(x)
    # jac      - name of the Jacobian function J(x)
    # x0       - initial guess for the solution x0
    # tol      - stopping tolerance for Newton's iteration
    # maxk     - maximum number of Newton iterations before stopping
    # fnonargs - optional arguments that will be passed to the nonlinear function (useful for additional function parameters)

    k = 0
    x = x0

    F = eval(fnon)(x,*fnonargs)
    n = F.size

    print(' k    f(xk)')

    # Main Newton loop
    while (norm(F,2) > tol and k <= maxk):
        # Evaluate Jacobian matrix
        J = eval(jac)(x,n,fnon,F,*fnonargs)

        # Take Newton step by solving the tangent problem
        delta = np.linalg.solve(J,-F)
        x = x + delta

        F = eval(fnon)(x,*fnonargs)

        print('{0:2.0f}  {1:2.2e}'.format(k, norm(F,2)))
        k += 1

    if (k >= maxk):
        print('Not converged')
    else:
        return x

Provide the initial value here:

In [2]:
import numpy as np

def initialData( m ):

  # Initial solution profile
  # Input: grid size m
  # Output: initial condition u

  # spatial domain endpoints
  xL = ...
  xR = ...

  h = (xR - xL) / (m-1)                      # mesh size
  x = np.linspace(xL, xR, m).reshape((m, 1)) # grid points

  v = ...

  # Boundary conditions
  uL = ...
  uR = ...

  return v

Implement the Burgers equations here:

In [3]:
def fun_burgers( u, uOld, dt, h ):

  # Nonlinear residual function
  # Input: u    - current solution values (dimension (m-2) x 1)
  #        uOld - previous solution (dimension m x 1)
  #        dt   - time-step
  #        h    - spatial grid size
  # Output: f   - nonlinear functions evaluated at u

  m = uOld.size
  f = np.zeros( (m-2, 1) )

  # Take the boundary conditions from the previous solution
  uL = uOld[0]
  uR = uOld[m-1]

  # Difference equation at the left boundary
  f[0] = ...

  # Difference equations at each internal node
  for i in range(1,m-3):
    f[i] = ...

  # Difference equation at the right boundary
  f[m-3] = ...

  return f

To simplify the Newton implementation, we will use the finite-difference Jacobian algorithm:

In [4]:
import numpy as np
import copy

def fdJacobian(x,n,fnon,F0,*fnonargs):
  J = np.zeros((n,n), dtype=np.float64)
  h = 10e-8

  for k in range(0,n):
    xb = copy.copy(x)
    xb[k] = xb[k] + h

    F = eval(fnon)(xb,*fnonargs)

    for i in range(0,n):
        J[i,k] = (F[i] - F0[i]) / h

  return J

Now we have everything we need to introduce the main time-stepping loop:

In [5]:
def burgersModel( dtIn, nTimeSteps, mIn ):

  # Implicit Euler in time, finite difference in space
  # numerical model for the Fisher Equation
  #  function sols = fisherModel( dtIn, nTimeSteps, mIn )
  # Input:  dtIn       - time step size
  #         nTimeSteps - number of time steps
  #         mIn        - grid dimension
  # Output: sols       - array of solution vectors u^k

  dt = dtIn   # time step size
  m = mIn     # dimension of spatial mesh for [X1,X2]
  n = m-2     # number of nonlinear equations (excluding boundary condition)
  h = 1/(m-1) # grid size

  # Initial and boundary conditions
  uOld = initialData( m )
  u0 = np.zeros( (n,1) )

  # Create an array for the solutions in time
  sols = []
  sols.append(uOld)

  # Time stepping loop
  for k in range( 0, nTimeSteps ):
    print('Time step: {0:1.0f}'.format(k))

    # Use previous time step as the Newton initial guess
    u0 = uOld[1:n+1]

    # Call the Newton solver
    u = myNewtonSys("fun_burgers", "fdJacobian", u0, 1e-6, 10, uOld, dt, h)

    # Update the previous time step
    uOld[1:n+1] = u[0:n]

    # Store the solution vector
    sols.append(copy.copy(uOld))

  return sols;

In [6]:
mPts = 50
tPts = 200
sols = burgersModel( 0.01, tPts, mPts )

TypeError: unsupported operand type(s) for -: 'ellipsis' and 'ellipsis'

We can visualise the solutions in one plot...

In [ ]:
from matplotlib import pyplot as plt
import numpy as np
%matplotlib inline
x=np.linspace(0, 1, mPts)

fig=plt.figure()
ax=fig.add_axes([0.0,0.0,1.0,1.0])
for t in range(0, tPts):
  ax.plot(x,sols[t])
ax.set_xlabel('x')
ax.set_ylabel('u(x,t)')

...or as an animation over time:

In [ ]:
from matplotlib import rc
rc('animation', html='jshtml')
from math import *
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
import matplotlib.animation as animation

fig = plt.figure(figsize=(8,6));
ax = plt.axes( );
plt.close();

In [ ]:
def frame(w):
    ax.clear()
    global x,y,z
    x=np.linspace(0, 1, mPts)
    ax.set_xlabel('x')
    ax.set_xlim(0.0, 1.0)
    ax.set_ylabel('u(x,t)')
    ax.set_ylim(0.0, 1.0)
    plot = ax.plot(x, sols[w])


In [ ]:
anim = animation.FuncAnimation(fig, frame, frames=tPts, blit=False, repeat=True)

In [ ]:
anim